In [2]:
import keyring        


In [1]:
import pymssql
from contextlib import contextmanager
@contextmanager
def GetDBConnection():
    conn = pymssql.connect(
        server="ITSMDB",
        database="BSC_4_0",
        as_dict=True
    )
    cursor = conn.cursor()
    try:
        yield cursor
    finally:
        conn.close()
        

In [9]:

@contextmanager
def GetDBConnection2():
    server = 'ITSMDB'
    database = 'BSC_4_0'
    username = 'ingham-county\\srudloff'  # Format: DOMAIN\\username
    password = keyring.get_password("VPN_Ingham-County",keyring.get_password("VPN_Ingham-County",'USER'))

    # Connect to the database
    conn = pymssql.connect(server, username, password, database,as_dict=True)

    cursor = conn.cursor()
    try:
        yield cursor
    finally:
        conn.close()
        

In [10]:
def findTicketByKeyword(keyword):
    with GetDBConnection2() as cursor:
        sql = """
        SELECT
        [Id],
        [Title],
        [Description]
        FROM [Tickets]
        where Description like %s
        or Title like %s;
        """
        cursor.execute(sql, (keyword,keyword))
        return cursor.fetchall()

In [11]:
def SearchComments(keyword):
    with GetDBConnection2() as cursor:
        sql = """
        SELECT
        [TicketId],
        [Body]
        FROM [Comments]
        where Body like %s
        """
        cursor.execute(sql, keyword)
        data = cursor.fetchmany(10)
        while data is not None:
            yield data
            data = cursor.fetchmany(10)

In [12]:
for i in findTicketByKeyword("%Malware%"):
    print(f"{i['Id']}: {i['Title']}")

50566: 50566 - PPZ - Malware on PPZ PC
51170: Investigate Malware
51948: Investigate Malware
52129: Thumb drive risk - 
52144: Investigate Malware
52175: Needs Software Deleted - Mike Allen
51146: Investigate Malware
51569: Investigate Malware
51752: Investigate Malware
53864: Investigate Malware
57384: Check email to determine if its malicious
57519: Investigate Malware
57708: Scan emails for malware
57778: Investigate Malware
57982: Investigate Malware
53912: Investigate Malware
53982: Investigate Malware
56125: Malware
56479: Investigate Malware
58513: Investigate Malware
58514: Investigate Malware
58642: Investigate Malware
59699: Mennythanks virus?
59932: Link clicked in spam email
57983: Investigate Malware
57987: Investigate Malware
60311: Investigate Malware - Amy Prieskorn
60362: Investigate Malware - IE popup about firewall issues
60439: Outlook issues and popups
61042: Investigate Malware
61124: Malwarebytes  keeps popping up on my screen, needs to be removed - Noreen Allman

In [15]:
for i in SearchComments("%Samuel Rudloff%"):
    print(f"{i['TicketId']}: {i['Body']}")

OperationalError: (20009, b'DB-Lib error message 20009, severity 9:\nUnable to connect: Adaptive Server is unavailable or does not exist (ITSMDB)\n')